In [111]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [112]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [113]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(13937, 8) (3485, 8) (13937,) (3485,)


In [114]:
# X 값 Window Dataset 구성
ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(WINDOW_SIZE, stride=1, shift=1, drop_remainder=True)  # 여기서는 X값만 별도로 구성하므로 WINDOW_SIZE 값에 1을 더하지 않는다!
ds_x = ds_x.flat_map(lambda x : x.batch(WINDOW_SIZE))

In [115]:
# y 값 Window Dataset 구성
ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[WINDOW_SIZE:])

In [116]:
train_data = tf.data.Dataset.zip((ds_x, ds_y)).batch(BATCH_SIZE)

In [117]:
for x, y in train_data.take(1) :
  print(x[:3])
  print()
  print(y[:3])

tf.Tensor(
[[[-0.11352868 -0.09386187 -0.6825146   0.09445245 -0.66581843
   -1.23511514 -0.21825161 -1.53935952]
  [-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]]

 [[-0.11352868 -0.9225242  -1.06038321  0.4051263  -0.66581843
   -1.23511514 -0.21825161 -1.57681737]
  [-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]]

 [[-0.11352868 -1.02610699 -1.06038321  0.53827224 -0.66581843
   -1.23511514 -0.21825161 -1.62925834]
  [-0.11352868 -1.12968978 -1.06038321  0.71580015 -0.66581843
   -1.23511514 -0.21825161 -1.65922462]
  [-0.11352868 -0.9225242  -1.06038321  0.80456411 -0.66581843
   -1.23511514 -0.21825161 -1.68919089]]], shape=(3, 3, 8), dtype=float64)

tf.Tensor([0. 0. 0.], 

In [118]:
pd.concat([SNE_X_train, SNE_y_train], axis=1).head(10)

,0,1,2,3,4,5,6,7,8
0,-0.113529,-0.093862,-0.682515,0.094452,-0.665818,-1.235115,-0.218252,-1.539360,0.0
1,-0.113529,-0.922524,-1.060383,0.405126,-0.665818,-1.235115,-0.218252,-1.576817,0.0
2,-0.113529,-1.026107,-1.060383,0.538272,-0.665818,-1.235115,-0.218252,-1.629258,0.0
3,-0.113529,-1.129690,-1.060383,0.715800,-0.665818,-1.235115,-0.218252,-1.659225,0.0
4,-0.113529,-0.922524,-1.060383,0.804564,-0.665818,-1.235115,-0.218252,-1.689191,0.0
5,-0.113529,-1.026107,-1.060383,0.937710,-0.665818,-1.235115,-0.218252,-1.719157,0.0
6,-0.113529,-0.922524,-1.060383,0.893328,-0.665818,-1.235115,-0.218252,-1.734140,0.0
7,-0.113529,-0.922524,-1.060383,0.982092,0.549517,-1.235115,-0.333181,-1.741632,6000.0
8,-0.113529,-1.129690,-1.060383,0.538272,1.764852,0.769233,-0.226741,-1.636750,16000.0
9,-0.113529,-0.922524,-1.060383,0.005688,1.764852,-0.984572,-0.322080,-1.329596,24000.0


In [119]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(SNE_X_train).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(SNE_y_train[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [120]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [121]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [122]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [123]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [124]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [125]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [126]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [127]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500


    494/Unknown - 7s 6ms/step - loss: 7618.4927 - mse: 183615760.0000 - mae: 7618.8862
Epoch 1: val_loss improved from inf to 7605.09668, saving model to new_multi_stne_lstm_weight.h5
498/498 [==============================] - 9s 10ms/step - loss: 7620.5151 - mse: 183582320.0000 - mae: 7620.9082 - val_loss: 7605.0967 - val_mse: 182927088.0000 - val_mae: 7605.5034 - lr: 0.0010
Epoch 2/500
491/498 [============================>.] - ETA: 0s - loss: 7577.9819 - mse: 181854624.0000 - mae: 7578.3555
Epoch 2: val_loss improved from 7605.09668 to 7542.90918, saving model to new_multi_stne_lstm_weight.h5
498/498 [==============================] - 4s 8ms/step - loss: 7577.9961 - mse: 181737680.0000 - mae: 7578.3706 - val_loss: 7542.9092 - val_mse: 180351248.0000 - val_mae: 7543.3066 - lr: 0.0010
Epoch 3/500
496/498 [============================>.] - ETA: 0s - loss: 7508.2588 - mse: 178852160.0000 - mae: 7508.6372
Epoch 3: val_loss improved from 7542.90918 to 7455.45996, saving model to new_multi

In [128]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500


    497/Unknown - 4s 3ms/step - loss: 7619.2817 - mse: 183641264.0000 - mae: 7619.6587
Epoch 1: val_loss improved from inf to 7606.37305, saving model to new_multi_stne_rnn_weight.h5
498/498 [==============================] - 5s 5ms/step - loss: 7621.5181 - mse: 183636368.0000 - mae: 7621.8945 - val_loss: 7606.3730 - val_mse: 183009968.0000 - val_mae: 7606.7886 - lr: 0.0010
Epoch 2/500
490/498 [============================>.] - ETA: 0s - loss: 7601.8623 - mse: 182581040.0000 - mae: 7602.2261
Epoch 2: val_loss improved from 7606.37305 to 7543.43506, saving model to new_multi_stne_rnn_weight.h5
498/498 [==============================] - 2s 4ms/step - loss: 7581.9614 - mse: 181844064.0000 - mae: 7582.3262 - val_loss: 7543.4351 - val_mse: 180302832.0000 - val_mae: 7543.8159 - lr: 0.0010
Epoch 3/500
484/498 [============================>.] - ETA: 0s - loss: 7487.6987 - mse: 178524208.0000 - mae: 7488.0649
Epoch 3: val_loss improved from 7543.43506 to 7425.84277, saving model to new_multi_st

In [129]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
    490/Unknown - 9s 7ms/step - loss: 7604.4023 - mse: 183099328.0000 - mae: 7604.7783
Epoch 1: val_loss improved from inf to 7611.61035, saving model to new_multi_stne_gru_weight.h5
498/498 [==============================] - 11s 11ms/step - loss: 7622.7158 - mse: 183703328.0000 - mae: 7623.0918 - val_loss: 7611.6104 - val_mse: 183226960.0000 - val_mae: 7611.9556 - lr: 0.0010
Epoch 2/500
495/498 [============================>.] - ETA: 0s - loss: 7600.7798 - mse: 182844896.0000 - mae: 7601.1504
Epoch 2: val_loss improved from 7611.61035 to 7570.00781, saving model to new_multi_stne_gru_weight.h5
498/498 [==============================] - 4s 8ms/step - loss: 7593.6943 - mse: 182465920.0000 - mae: 7594.0645 - val_loss: 7570.0078 - val_mse: 181515104.0000 - val_mae: 7570.3750 - lr: 0.0010
Epoch 3/500
497/498 [============================>.] - ETA: 0s - loss: 7538.6333 - mse: 180214848.0000 - mae: 7539.0044
Epoch 3: val_loss improved from 7570.00781 to 7522.91016, saving model t

In [146]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7251948118209839
lstm_history Validation MSE: 16.099933624267578
lstm_history Validation MAE: 2.095870018005371
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.8077667951583862
lstm_history_ns Validation MSE: 17.04073143005371
lstm_history_ns Validation MAE: 2.169351816177368
-------------rnn_history-------------
rnn_history Validation Loss: 1.7638643980026245
rnn_history Validation MSE: 16.553455352783203
rnn_history Validation MAE: 2.133317708969116
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.820894718170166
rnn_history_ns Validation MSE: 17.102962493896484
rnn_history_ns Validation MAE: 2.1736433506011963
-------------gru_history-------------
gru_history Validation Loss: 1.7141441106796265
gru_history Validation MSE: 16.135080337524414
gru_history Validation MAE: 2.0728256702423096
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 

In [131]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [132]:
# Feature와 Label 분리하기
def Feature_Label(datafile) :
    X = datafile.iloc[:,:-1]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 8) (17422,)
(42062, 8) (10516, 8) (42062,) (10516,)


In [133]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [134]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [135]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 8), (28,)


In [136]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [137]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [138]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 8]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [139]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [140]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [141]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500


   1502/Unknown - 16s 7ms/step - loss: 1.9344 - mse: 19.7891 - mae: 2.2939
Epoch 1: val_loss improved from inf to 2.03408, saving model to base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 18s 8ms/step - loss: 1.9346 - mse: 19.7907 - mae: 2.2941 - val_loss: 2.0341 - val_mse: 19.5581 - val_mae: 2.4176 - lr: 0.0010
Epoch 2/500
1503/1503 [==============================] - ETA: 0s - loss: 1.3564 - mse: 9.9472 - mae: 1.7096
Epoch 2: val_loss did not improve from 2.03408
1503/1503 [==============================] - 11s 7ms/step - loss: 1.3564 - mse: 9.9472 - mae: 1.7096 - val_loss: 2.0619 - val_mse: 20.1536 - val_mae: 2.4382 - lr: 0.0010
Epoch 3/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.2932 - mse: 9.2488 - mae: 1.6374
Epoch 3: val_loss improved from 2.03408 to 2.00100, saving model to base_multi_stne_lstm_weight.h5
1503/1503 [==============================] - 11s 8ms/step - loss: 1.2938 - mse: 9.2515 - mae: 1.6381 - val_loss: 2.0010 - val_m

In [142]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1481/Unknown - 5s 2ms/step - loss: 1.9690 - mse: 21.0225 - mae: 2.3431
Epoch 1: val_loss improved from inf to 2.06449, saving model to base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 6s 3ms/step - loss: 1.9701 - mse: 20.9767 - mae: 2.3445 - val_loss: 2.0645 - val_mse: 19.9420 - val_mae: 2.4750 - lr: 0.0010
Epoch 2/500
1493/1503 [============================>.] - ETA: 0s - loss: 1.3506 - mse: 9.8178 - mae: 1.7002
Epoch 2: val_loss improved from 2.06449 to 1.97660, saving model to base_multi_stne_rnn_weight.h5
1503/1503 [==============================] - 4s 3ms/step - loss: 1.3551 - mse: 9.8746 - mae: 1.7047 - val_loss: 1.9766 - val_mse: 18.5601 - val_mae: 2.3728 - lr: 0.0010
Epoch 3/500
1487/1503 [============================>.] - ETA: 0s - loss: 1.3163 - mse: 9.4563 - mae: 1.6620
Epoch 3: val_loss did not improve from 1.97660
1503/1503 [==============================] - 4s 3ms/step - loss: 1.3214 - mse: 9.5302 - mae: 1.6673 - val_loss: 2.0613 -

In [143]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1495/Unknown - 14s 6ms/step - loss: 1.9551 - mse: 21.0619 - mae: 2.3148
Epoch 1: val_loss improved from inf to 2.01522, saving model to base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 15s 7ms/step - loss: 1.9558 - mse: 21.0812 - mae: 2.3155 - val_loss: 2.0152 - val_mse: 19.1159 - val_mae: 2.4001 - lr: 0.0010
Epoch 2/500
1502/1503 [============================>.] - ETA: 0s - loss: 1.3271 - mse: 9.5259 - mae: 1.6775
Epoch 2: val_loss did not improve from 2.01522
1503/1503 [==============================] - 10s 6ms/step - loss: 1.3270 - mse: 9.5253 - mae: 1.6774 - val_loss: 2.3967 - val_mse: 25.4663 - val_mae: 2.7746 - lr: 0.0010
Epoch 3/500
1497/1503 [============================>.] - ETA: 0s - loss: 1.2643 - mse: 8.9409 - mae: 1.6075
Epoch 3: val_loss improved from 2.01522 to 1.88929, saving model to base_multi_stne_gru_weight.h5
1503/1503 [==============================] - 10s 6ms/step - loss: 1.2652 - mse: 8.9569 - mae: 1.6085 - val_loss: 1.88

In [144]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.7251948118209839
lstm_history Validation MSE: 16.099933624267578
lstm_history Validation MAE: 2.095870018005371
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 1.8077667951583862
lstm_history_ns Validation MSE: 17.04073143005371
lstm_history_ns Validation MAE: 2.169351816177368
-------------rnn_history-------------
rnn_history Validation Loss: 1.7638643980026245
rnn_history Validation MSE: 16.553455352783203
rnn_history Validation MAE: 2.133317708969116
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 1.820894718170166
rnn_history_ns Validation MSE: 17.102962493896484
rnn_history_ns Validation MAE: 2.1736433506011963
-------------gru_history-------------
gru_history Validation Loss: 1.7141441106796265
gru_history Validation MSE: 16.135080337524414
gru_history Validation MAE: 2.0728256702423096
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 